## Course 2:  Symbolic Execution
### **Exercise 1.** Practical Symbolic Execution using Manticore

The purpose of this exercise is to equip you with hands-on experience of applying *existing* symbolic execution tools to analyze smart contracts.
This exercise is based on [Manticore](https://github.com/trailofbits/manticore)—a symbolic execution engine developed in Python by [Trail of Bits](https://www.trailofbits.com/). *It is also inspired by the Manticore's own [tutorial](https://ethereum.org/en/developers/tutorials/how-to-use-manticore-to-find-smart-contract-bugs/).*
This Jupyter notebook uses Python 3.7.

#### **Installation**

Manticore (and Solidity compiler solc) can be installed using `pip`.

In [ ]:
!pip install "manticore[native]" solc-select
!solc-select install 0.5.11
!solc-select use 0.5.11

#### **Manticore Basics**

In this exercise, we will be using Python API provided by Manticore. Using the API commands, we can write a Python script controlling the symbolic execution process. First, the script should instantiate a new blockchain `m`:

In [ ]:
from manticore.ethereum import ManticoreEVM

m = ManticoreEVM()

Now let's use Manticore to analyze the smart contract used as an example in the [symbolic execution overview](https://github.com/baolean/formal-methods-curriculum/blob/master/courses/2_Approaches_Modeling_Verification/content/1_Symbolic_Execution/symbolic_execution.md). The next cell loads smart contract's source code into the `source_code` variable:

In [ ]:
source_code = """
contract Exchange {   
    function deposit(uint _in, uint _out, uint _approved) public {
        uint balance_in = 100; uint balance_out = 100;

        if (_in >= _out && _out <= balance_out) {
            if (_approved >= _in) {
                balance_in += _in;
                _approved -= _in;
            }
            
            balance_out -= _out;
        }

        assert(balance_in + balance_out >= 200);
   }
}
"""

To analyze a smart contract, it has to be deployed and, then, called via a transaction. To do so, we need to set up a regular (non-smart-contract) user account, which can be created using `m.create_account` command provided by the Manticore API. It can be called with [several parameters](https://manticore.readthedocs.io/en/latest/evm.html?highlight=create_account#manticore.ethereum.ManticoreEVM.create_account) specifying its balance, address, etc.

A smart contract defined in `source_code` can be, then, deployed using `m.solidity_create_contract` function. As parameters, we'll pass the smart contract code and the owner address we have just created. Other optional parameters can be found in the [documentation](https://manticore.readthedocs.io/en/latest/evm.html?highlight=create_account#manticore.ethereum.ManticoreEVM.create_contract). The script, then, becomes:

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

Manticore supports *raw* and *named* transactions. A *raw* transaction explores all smart contract functions—the transaction caller, data, address, or value can all be symbolic. However, in this example we will use *named* transactions which explore only one particular function, but it can do so with *symbolic* input arguments.

In Manticore API, symbolic variables (normally a uint256) are created using [`m.make_symbolic_value`](https://manticore.readthedocs.io/en/latest/evm.html?highlight=create_account#manticore.ethereum.ManticoreEVM.make_symbolic_value).
To call a function of a smart contract, run `contract_account.func_name(arguments)`.

**Task 1:** declare three symbolic variables representing `_in`, `_out`, and `_approved`, and call a `deposit(_in, _out, _approved)` function with these symbolic arguments.

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

# TODO: 
# (1) declare three symbolic variables representing `_in`, `_out`, and `_approved`
# (2) call a `deposit(_in, _out, _approved)` function with these symbolic arguments.

In [ ]:
#@title Task 1 Solution 

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr);

Now let's perform the actual analysis of the smart contract and its `balance_in + balance_out >= 200` assertion.
As a symbolic execution tool, Manticore systematically analyses all possible execution paths. Each path ends in a certain blockchain state. The list of all final states can be accessed using `m.all_states`. In this exercise, we are interested in paths that end with a `THROW` (or `REVERT`) instruction, meaning that the execution was terminated—possibly, by a failed asserion. These states can be obtained via `m.terminated_states`.
Now, the concrete values corresponding to a state can be obtained using `m.generate_testcase(state, name="TestCaseName")`.
The results would be saved into a workplace directory `m.workspace`—you can find it in the `Files` tab in Colab.

**Task 2:** obtain the list of terminated states and generate a test case for each of them.

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

# TODO: 
# (1) declare three symbolic variables representing `_in`, `_out`, and `_approved`
# (2) call a `deposit(_in, _out, _approved)` function with these symbolic arguments.

# (3) obtain a list of terminated states
# (4) generate a test case for each of them

print(f'Assertion violation found, results are in {m.workspace}')

Assert violation found, results are in /content/mcore_xb87eq5q


In [ ]:
#@title Task 2 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr);

for state in m.terminated_states:
      m.generate_testcase(state, name="AssertViolated")
      print(f'Assertion violation found, results are in {m.workspace}')


If we review the generated test cases, we will notice that two test cases are generated. After the manual symbolic execution performed in the write-up, we are aware of *one* logical issue in the smart contract that might cause an assertion to fail (`_in >= _out && _approve < in`). If we look at the concrete values generated by Manticore (they can be found in `AssertViolated_00000000.tx` and `AssertViolated_00000001.tx`), we'll notice that these values are, in fact, quite big, which might suggest that the failure is caused by integer underflow or overflow (it is). Now, for training purposes, let's assume that the caller can only use relatively small values as arguments when calling `deposit`—for example, less than or equal to 200.
*(Please **never** make such assumptions in practice)*.

This assumption can be expressed as a constraint (`m.constrain(...)`) on the symbolic values we declared earlier. The constraint itself is an expression that can be constructed from a set of Operators available in Manticore. To import the Operators to your script, run `from manticore.core.smtlib import Operators`. Operators and their implementation can be found [here](https://github.com/trailofbits/manticore/blob/master/manticore/core/smtlib/operators.py). A *less-than-or-equal* Boolean expression corresponds to `Operators.ULE(operand1, operand2)` (`U` in `ULE` stands for `unsigned`—the details on that might/will be covered in next exercises).

**Task 3**: Add upper-bound value constraints (e.g., <= 200) on all three symbolic variables.

In [ ]:
from manticore.core.smtlib import Operators

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

# TODO: 
# (1) declare three symbolic variables representing `_in`, `_out`, and `_approved`
# (5) add constraints on symbolic variables

# (2) call a `deposit(_in, _out, _approved)` function with these symbolic arguments.

# (3) obtain a list of terminated states
# (4) generate a test case for each of them

print(f'Assertion violation found, results are in {m.workspace}')

In [ ]:
#@title Task 3 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

m.constrain(Operators.ULE(symbolic_in, 200))
m.constrain(Operators.ULE(symbolic_out, 200))
m.constrain(Operators.ULE(symbolic_appr, 200))

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr);

for state in m.terminated_states:
      m.generate_testcase(state, name="BugFound")
      print(f'Assertion violation found, results are in {m.workspace}')

Although now we only generate a single expected test case, the assumptions we made on variable values are not realistic. Let's add overflow protection to the smart contract source code and remove the `assert` from the code (since it can overflow too). Instead, let's encode this requirement as a property in our Python script. Let's re-write the example smart contract source code by adding overflow protection to `balance_in += _in;` and removing the `assert` statement. A common way to protect from integer overflow is to check that the following condition holds: *the result of a sum is greater than or equal to the (first) operand*. In our example, it translates into `require(balance_in >= _in)`.

In [ ]:
source_code = """
contract Exchange {   
    function deposit(uint _in, uint _out, uint _approved) public {
        uint balance_in = 100; uint balance_out = 100;

        if (_in >= _out && _out <= balance_out) {
            if (_approved >= _in) {
                balance_in += _in;
                require(balance_in >= _in);

                _approved -= _in;
            }
            
            balance_out -= _out;
        }
   }
}
"""

Since now `assert` has been removed and overflowing executions revert, we need to analyze paths that executed successfuly: we can get them using `m.ready_states`.

We also need to define the property we'll be checking: that the sum of `balance_in` and `balance_out` should not decrease. In other words, we want to identify the \[vulnerable] states where this sum of balances after `deposit()` is *less* than the sum of balances before. We can identify these states by adding a corresponding constraint, or `condition`, by adding it to `m.generate_testcase(state, name="BugFound", only_if=condition)`.

However, we need to formulate the property as an expression that Manticore understands. To do so, we need to obtain the values of `balance_in` and `balance_out` before and after the execution.

Let's make the code more realistic by assuming that `balance_in` and `balance_out` are public variables with their values being set to 100 in the constructor.
In this case, their values are returned by the following function calls: `contract_account.balance_in()` and `contract_account.balance_out()`—these getter functions are automatically generated by the Solidity compiler.

In [ ]:
source_code = """
contract Exchange {   
    uint public balance_in;
    uint public balance_out;

    constructor() public {
        balance_in = 100;
        balance_out = 100;
    }

    function deposit(uint _in, uint _out, uint _approved) public {
        if (_in >= _out && _out <= balance_out) {
            if (_approved >= _in) {
                balance_in += _in;
                require(balance_in >= _in);

                _approved -= _in;
            }
            
            balance_out -= _out;
        }
   }
}
"""


**Task 4**: Instrument the script with four additional transactions that return smart contract balances (two before `deposit`, two after). Iterate through states that correspond to successful (not terminated) executions to check that the script works as expected.

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

# (1) Get values of balance_in and balance_out *before* deposit()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out);

# (2) Get values of balance_in and balance_out *after* deposit()

for state in # (3) iterate over states that are *ready*
  print(state)

print(f'States info can be found in {m.workspace}')

In [ ]:
#@title Task 4 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.balance_in()
contract_account.balance_out()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr);

contract_account.balance_in()
contract_account.balance_out()

for state in m.ready_states:
  print(state)

print(f'States info can be found in {m.workspace}')


To construct a property, we have to process the result returned by the `balance_in()` and `balance_out()` functions. The (serialized) data returned by a transaction *in a state* can be obtained using `state.platform.transactions[n].return_data` command, where `n` corresponds to the position of a transaction in a sequence of calls in the analysis script. For example, our script now has the following transactions and indices:

\[0] — m.solidity_create_contract

\[1] — contract_account.balance_in()

\[2] — contract_account.balance_out()

\[3] — contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr)

...and so on.

As you may notice, the values of variables we're interested in are returned by transactions 1, 2, 4, and 5. To be used in an expression, we need to deserialize the return data into an unsigned integer: this can be done by `ABI.deserialize("uint", return_data)`.
`ABI` needs to be imported prior to usage:
`from manticore.ethereum import ABI`.

After the balances before and after `deposit` are obtained, we can define a constraint that captures the vulnerable state criterion. For illustration purposes, let's make the requirement even more strict: the sum of balances in a smart contract should only increase. The vulnerable condition, then, becomes the following: the sum of balances *before* the function execution is *greater than or equal to* the sum *after*. This can be formulated using the *unsigned greater than* operator: `Operators.UGT(operand1, operand2)`.

**Task 5**: Process the data returned by `balance_in()` and `balance_out()`  calls. Use the obtained values to build a condition for test case generation, pass this condition to `m.generate_testcase` as an `only_if` argument.

In [ ]:
from manticore.ethereum import ABI

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

# (1) Get values of balance_in and balance_out *before* deposit()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr)

# (2) Get values of balance_in and balance_out *after* deposit()

for state in m.ready_states:
    # (3) Get balance_in and balance_out *before* deposit() and sum them up
    # (4) Get balance_in and balance_out *after* deposit() and sum them up

    condition = # (5) define a condition that detects a vulnerable state
    
    # (6) pass the condition as only_if=... parameter in generate_testcase
    if m.generate_testcase(state, name="BugFound"):
        print(f'Assertion violation found, results are in {m.workspace}')

In [ ]:
#@title Task 5 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.balance_in()
contract_account.balance_out()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr)

contract_account.balance_in()
contract_account.balance_out()

for state in m.ready_states:
    in_balance_before = state.platform.transactions[1].return_data
    in_balance_before = ABI.deserialize("uint", in_balance_before)

    out_balance_before = state.platform.transactions[2].return_data
    out_balance_before = ABI.deserialize("uint", out_balance_before)

    in_balance_after = state.platform.transactions[4].return_data
    in_balance_after = ABI.deserialize("uint", in_balance_after)

    out_balance_after = state.platform.transactions[5].return_data
    out_balance_after = ABI.deserialize("uint", out_balance_after)

    sum_before = in_balance_before + out_balance_before
    sum_after = in_balance_after + out_balance_after

    condition = Operators.UGT(sum_before, sum_after)

    if m.generate_testcase(state, name="BugFound", only_if=condition):
        print(f'Assertion violation found, results are in {m.workspace}')

We can, however, notice that two test cases are generated. One of them is, again, attributed to integer overflow that occurs when we sum `balance_in` and `balance_out` before (and after) the execution. Since these operations are not part of the analyzed smart contract, let's add an additional "no-overflow" constraint to the test case generation condition.
Our final condition will consist of two parts: *the sum of balances did not increase AND no overflow occured during the (balance_in + balance_out) operation*.
To implement AND, use `Operators.AND`. In the absence of integer overflow, the following condition should hold: *the result of a sum is greater than or equal (`Operators.UGT`) to the first operand*.

(Final) **Task 6** Integrate the no-overflow-on-a-sum constraint into a condition that is used to detect vulnerable state. Analyze the resulting test case, check if it matches the result of the manual analysis performed in the write-up.

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

# (1) Get values of balance_in and balance_out *before* deposit()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr)

# (2) Get values of balance_in and balance_out *after* deposit()

for state in m.ready_states:
    # (3) Get balance_in and balance_out *before* deposit() and sum them up
    # (4) Get balance_in and balance_out *after* deposit() and sum them up

    # (5) Define a constraint that ensures that no overflow
    #      occurs during balances-before sum-up 
    # (6) Define a constraint that ensures that no overflow
    #      occurs during balances-after sum-up 


    condition = # (7) define a condition includes no-overflow
                #   and property violation constraints
    
    # (8) pass the condition as only_if=... parameter in generate_testcase
    if m.generate_testcase(state, name="BugFound"):
        print(f'Assertion violation, results are in {m.workspace}')


In [ ]:
#@title Task 6 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.balance_in()
contract_account.balance_out()

contract_account.deposit(symbolic_in, symbolic_out, symbolic_appr)

contract_account.balance_in()
contract_account.balance_out()

for state in m.ready_states:
    in_balance_before = state.platform.transactions[1].return_data
    in_balance_before = ABI.deserialize("uint", in_balance_before)

    out_balance_before = state.platform.transactions[2].return_data
    out_balance_before = ABI.deserialize("uint", out_balance_before)

    in_balance_after = state.platform.transactions[4].return_data
    in_balance_after = ABI.deserialize("uint", in_balance_after)

    out_balance_after = state.platform.transactions[5].return_data
    out_balance_after = ABI.deserialize("uint", out_balance_after)

    sum_before = in_balance_before + out_balance_before
    sum_after = in_balance_after + out_balance_after
    
    no_before_overflow = Operators.UGE(sum_before, in_balance_before)
    no_after_overflow = Operators.UGE(sum_after, in_balance_after)
    no_sum_overflow = Operators.AND(no_before_overflow, no_after_overflow)

    condition = Operators.AND(Operators.UGT(sum_before, sum_after),
                                no_sum_overflow)

    if m.generate_testcase(state, name="BugFound", only_if=condition):
        print(f'Assertion violation found, results are in {m.workspace}')